In [63]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import glob
import imageio
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [97]:
def processData(imPath, labelPath):
    df1 = pd.read_csv(labelPath, header=0)
    yEncoder = OneHotEncoder(handle_unknown='ignore')
    yEncoder.fit(df1[['label']])
    #labelNames = yEncoder.get_feature_names()
    labels = yEncoder.transform(df1[['label']]).toarray()
    df2 = pd.DataFrame(data=labels)
    df1.drop(['label'], axis=1, inplace=True)
    df = pd.concat([df1, df2], axis=1)
    record = df.set_index('id').T.to_dict('list')
    #print(record[3])
    data = []
    imPaths = glob.glob(imPath)
    for i in imPaths:
        #print(i.split('.')[1][7:])
        ID = int(i.split('.')[1][7:])
        tmp = [imageio.imread(i), record[ID]]
        data.append(tmp)
    return data

In [99]:
df = pd.read_csv('./trainLabels.csv', header=0)
df[['label']]
data = processData('./train/*.png', './trainLabels.csv')
#imPaths = glob.glob('./train/*.png')
#imPaths = sorted(imPaths, key=lambda x:x[10:-3])
#print(imPaths)
#images = np.array([])
#images = []
#for i in imPaths:
    #images.append(imageio.imread(i))
    #images = np.append(images, imageio.imread(i))
#print(len(images))
#print(images.shape)

In [111]:
print(len(data[5][0]))
print(imageio.imread('./train/1.png').shape)

32
(32, 32, 3)
